# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [8]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruitsDB', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [11]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65680145b9a4d60f27e34b0e'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

In [12]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant = {"BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"22 King William Walk",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE10 9HU",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.09384000",
        "latitude":"51.29014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True} 

In [14]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('656804de0e1fcce20c81e5aa'), acknowledged=True)

In [15]:
# Check that the new restaurant was inserted
query = {'BusinessName' : 'Penang Flavours'}
results = establishments.find(query)
for result in results: 
    print(result)

{'_id': ObjectId('656804de0e1fcce20c81e5aa'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '22 King William Walk', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE10 9HU', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.09384000', 'latitude': '51.29014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [16]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

# BusinessType and BusinessTypeID
field = {'BusinessType': 1, 'BusinessTypeID': 1,  '_id': 0}

results = establishments.find(query, field)

pprint (results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [17]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"BusinessName" : "Penang Flavours"},{'$set' : {"BusinessTypeID" : 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# Confirm that the new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}

field = {'BusinessName': 1, 'BusinessTypeID': 1,  '_id': 0}

results = establishments.find(query, field)

pprint (results[0])

{'BusinessName': 'Penang Flavours', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [21]:
# Find how many documents have LocalAuthorityName as "Dover"
queryDover = {"LocalAuthorityName" : "Dover"}

print("Number of documents:", establishments.count_documents(queryDover))

Number of documents: 994


In [22]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(queryDover)
results = establishments.find(queryDover)
for result in results:
    print(result)

In [24]:
# Check if any remaining documents include Dover
print("Number of documents:", establishments.count_documents(queryDover))

Number of documents: 0


In [25]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65680145b9a4d60f27e34df5'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [58]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ { '$set': {'geocode.latitude' : {'$toDouble': '$geocode.latitude'}}}])

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [62]:
establishments.update_many({}, [ { '$set': {'geocode.longitude' : {'$toDecimal': '$geocode.longitude'}}}])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [63]:
print(type(result['geocode']['latitude']))
print(type(result['geocode']['longitude']))

<class 'float'>
<class 'str'>


In [52]:
db.establishments.distinct('geocode')

[{'longitude': Decimal128('0.00000900000000000000'), 'latitude': 51.6868977},
 {'longitude': Decimal128('0.00000900000031833770'),
  'latitude': 51.686897277832},
 {'longitude': Decimal128('0.0000130436926169919'),
  'latitude': 51.6317453586187},
 {'longitude': Decimal128('0.0000370000000000000'), 'latitude': 51.46422},
 {'longitude': Decimal128('0.0000602000000000000'), 'latitude': 51.6076079},
 {'longitude': Decimal128('0.0000639999998384155'),
  'latitude': 51.5382308959961},
 {'longitude': Decimal128('0.0000710000000000000'), 'latitude': 51.513715},
 {'longitude': Decimal128('0.000113000000000000'), 'latitude': 51.607714},
 {'longitude': Decimal128('0.000118999996630009'),
  'latitude': 51.5394859313965},
 {'longitude': Decimal128('0.000141700000000000'), 'latitude': 51.5417094},
 {'longitude': Decimal128('0.000157000000000000'), 'latitude': 51.686883},
 {'longitude': Decimal128('0.000173500000000000'), 'latitude': 51.6088739},
 {'longitude': Decimal128('0.000178000000000000'), 'l

Use `update_many` to convert `RatingValue` to integer numbers.

In [27]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [29]:
# Change the data type from String to Integer for RatingValue
db.establishments.distinct('RatingValue')

# set to int
establishments.update_many({}, [ { '$set': {'RatingValue' : {'$toInt': '$RatingValue'}}}])


result = establishments.find_one()

# check results
print(type(result['RatingValue']))

db.establishments.distinct('RatingValue')

<class 'int'>


[None, 0, 1, 2, 3, 4, 5]

In [61]:
# Check that the coordinates and rating value are now numbers
print(type(result['geocode']['latitude']))
print(type(result['geocode']['longitude']))
print(type(result['RatingValue']))

<class 'float'>
<class 'str'>
<class 'int'>
